# River mask development using DSWE/Landsat

The following code uses the DWSE watermasking method and Landsat remote sensing data to develop two-dimensional water masks of rivers within the given study area, drawing on locally stored shapefiles for study boundaries and Google Earth Engine for Landsat atmospherically corrected surface reflectance data. The masks are cleaned using a buffer of the Global River Widths from Landsat (GRWL) database. The user must first copy the path to a standard RivMapper input sheet .csv. Within that input sheet, the user must populate the river name (river_name), working directory (working_directory), the reach length modulator (reach_length_modulator), the buffer width as a fraction of the reach length (buffer_width), the DSWE water detection threshold from 1 to 4 (DSWE threshold), the year and reach ranges (year_range and reach_range), mask output path (mask_output_path), GRWL directory (grwl_directory), and minimum number of contiguous pixels to keep the water blob (min_pixels). Executing the process_multiple_rivers() function generates median annual composite water masks for the for the specified range of years for the given watershed, exporting them as .tif files to a specified local folder.

Note: for the year range and reach range, exceptable input syntax may be a tuple {e.g. (1985, 2024) for the year or (1, 10) for the reach}, a single value {e.g. 2024 for the year or 10 for the reach}, or "All", which if entered for the year range will attempt to process the entire Landsat record (1985 - 2025) and if entered for the reach range will attempt to process all reaches for the specified river.

DSWE Methodology: 
Jones, J.W., 2019. Improved Automated Detection of Subpixel-Scale Inundation—Revised Dynamic Surface Water Extent (DSWE) Partial Surface Water Tests. Remote Sensing 11, 374. https://doi.org/10.3390/rs11040374

Landsat Collection 2:
Earth Resources Observation and Science (EROS) Center. (2020). Landsat 8-9 Operational Land Imager / Thermal Infrared Sensor Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9OGBGM6.
Earth Resources Observation and Science (EROS) Center. (2020). Landsat 7 Enhanced Thematic Mapper Plus Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9C7I13B.
Earth Resources Observation and Science (EROS) Center. (2020). Landsat 4-5 Thematic Mapper Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9IAXOVV.

Google Earth Engine:
Gorelick, N., Hancher, M., Dixon, M., Ilyushchenko, S., Thau, D., Moore, R., 2017. Google Earth Engine: Planetary-scale geospatial analysis for everyone. Remote Sensing of Environment, Big Remotely Sensed Data: tools, applications and experiences 202, 18–27. https://doi.org/10.1016/j.rse.2017.06.031

Code adapted from Dr. Evan Greenberg: https://github.com/evan-greenbrg
Greenberg, E., Chadwick, A.J., Ganti, V., 2023. A Generalized Area-Based Framework to Quantify River Mobility From Remotely Sensed Imagery. Journal of Geophysical Research: Earth Surface 128, e2023JF007189. https://doi.org/10.1029/2023JF007189

Author: James (Huck) Rees;
        PhD Student, UCSB Geography
        
Date: January 12, 2025

## Import packages

In [1]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
import os
from datetime import datetime
import requests
import zipfile
import io
import math
import ast
import time
ee.Initialize()

import rasterio
from rasterio.merge import merge
from rasterio.transform import from_origin
from rasterio.features import geometry_mask, rasterize
from shapely.geometry import shape, Polygon
from scipy.ndimage import label
import numpy as np
import matplotlib.pyplot as plt

import glob
import subprocess
import ast
import re

## Initialize functions for DSWE water masking

In [2]:
# Normalized Difference Water Index (MNDWI)
def Mndwi(image):
    """
    Calculate the Modified Normalized Difference Water Index (MNDWI) for a given image.

    Parameters:
    image (ee.Image): The input image.

    Returns:
    ee.Image: The resulting image with the MNDWI band named 'mndwi'.
    """
    return image.normalizedDifference(['Green', 'Swir1']).rename('mndwi')

# Modified Bare Soil Reflectance Variables
def Mbsrv(image):
    """
    Calculate the Modified Bare Soil Reflectance Variable (MBSRV) for a given image.

    Parameters:
    image (ee.Image): The input image.

    Returns:
    ee.Image: The resulting image with the MBSRV band named 'mbsrv'.
    """
    return image.select(['Green']).add(image.select(['Red'])).rename('mbsrv')

def Mbsrn(image):
    """
    Calculate the Modified Bare Soil Reflectance Normalized (MBSRN) for a given image.

    Parameters:
    image (ee.Image): The input image.

    Returns:
    ee.Image: The resulting image with the MBSRN band named 'mbsrn'.
    """
    return image.select(['Nir']).add(image.select(['Swir1'])).rename('mbsrn')

# Normalized Difference Vegetation Index (NDVI)
def Ndvi(image):
    """
    Calculate the Normalized Difference Vegetation Index (NDVI) for a given image.

    Parameters:
    image (ee.Image): The input image.

    Returns:
    ee.Image: The resulting image with the NDVI band named 'ndvi'.
    """
    return image.normalizedDifference(['Nir', 'Red']).rename('ndvi')

# Automated Water Extraction Index (AWESH)
def Awesh(image):
    """
    Calculate the Automated Water Extraction Index (AWEsh) for a given image.

    Parameters:
    image (ee.Image): The input image with the necessary bands for MBSRN calculation.

    Returns:
    ee.Image: The resulting image with the AWEsh band named 'awesh'.
    """
    return image.expression(
        'Blue + 2.5 * Green + (-1.5) * mbsrn + (-0.25) * Swir2',
        {
            'Blue': image.select(['Blue']),
            'Green': image.select(['Green']),
            'mbsrn': Mbsrn(image).select(['mbsrn']),
            'Swir2': image.select(['Swir2'])
        }
    ).rename('awesh')

# Decision Tree for Surface Water Extent (DSWE)
def Dswe(image):
    """
    Calculate the Decision Tree for Surface Water Extent (DSWE) for a given image.

    Parameters:
    image (ee.Image): The input image with bands required for the DSWE calculation.

    Returns:
    ee.Image: The resulting image with the DSWE classification named 'dswe'.
    """
    mndwi = Mndwi(image)
    mbsrv = Mbsrv(image)
    mbsrn = Mbsrn(image)
    awesh = Awesh(image)
    swir1 = image.select(['Swir1'])
    nir = image.select(['Nir'])
    ndvi = Ndvi(image)
    blue = image.select(['Blue'])
    swir2 = image.select(['Swir2'])

    # Decision tree thresholds
    t1 = mndwi.gt(0.124)
    t2 = mbsrv.gt(mbsrn)
    t3 = awesh.gt(0)
    t4 = (mndwi.gt(-0.44)
          .And(swir1.lt(0.09))
          .And(nir.lt(0.15))
          .And(ndvi.lt(0.7)))
    t5 = (mndwi.gt(-0.5)
          .And(blue.lt(0.1))
          .And(swir1.lt(0.3))
          .And(swir2.lt(0.1))
          .And(nir.lt(0.25)))

    # Combine results using weights to create unique classes
    t = t1.add(t2.multiply(10)).add(t3.multiply(100)).add(t4.multiply(1000)).add(t5.multiply(10000))

    # Define DSWE classification levels
    noWater = t.eq(0).Or(t.eq(1)).Or(t.eq(10)).Or(t.eq(100)).Or(t.eq(1000))
    hWater = t.eq(1111).Or(t.eq(10111)).Or(t.eq(11011)).Or(t.eq(11101)).Or(t.eq(11110)).Or(t.eq(11111))
    mWater = (t.eq(111).Or(t.eq(1011)).Or(t.eq(1101)).Or(t.eq(1110))
              .Or(t.eq(10011)).Or(t.eq(10101)).Or(t.eq(10110))
              .Or(t.eq(11001)).Or(t.eq(11010)).Or(t.eq(11100)))
    pWetland = t.eq(11000)
    lWater = (t.eq(11).Or(t.eq(101)).Or(t.eq(110)).Or(t.eq(1001))
              .Or(t.eq(1010)).Or(t.eq(1100)).Or(t.eq(10000))
              .Or(t.eq(10001)).Or(t.eq(10010)).Or(t.eq(10100)))

    # Assign classification levels to DSWE
    iDswe = (noWater.multiply(0)
             .add(hWater.multiply(1))
             .add(mWater.multiply(2))
             .add(pWetland.multiply(3))
             .add(lWater.multiply(4)))

    return iDswe.rename(['dswe'])

# Generate Binary Water Mask Based on DSWE
def ClassifyWaterJones2019(image, max_level):
    """
    Creates a binary water mask from an input image using DSWE classification.
    
    Args:
        image: The input image containing relevant bands and indices.
        max_level: The maximum classification level (inclusive) to include in the mask.
    
    Returns:
        ee.Image: A single-band image representing the binary water mask.
    """
    dswe = Dswe(image)
    # Start with level 1 (high-confidence water)
    water_mask = dswe.eq(1)

    # Add higher levels if within the specified max level
    for level in range(2, max_level + 1):
        water_mask = water_mask.Or(dswe.eq(level))

    return water_mask.rename(['waterMask'])

## Initialize functions to curate Landsat imagery from GEE (collection 2)

In [3]:
def maskL8sr(image):
    """
    Masks out clouds and cloud shadows within Landsat 8 and 9 images using the BQA band.

    Args:
        image: ee.Image, the input Landsat image.

    Returns:
        ee.Image: The cloud-masked image.
    """
    # Bit positions for cloud shadow (bit 3) and clouds (bit 5) in the QA_PIXEL band
    cloud_shadow_bit_mask = (1 << 3)
    clouds_bit_mask = (1 << 5)
    # Select the BQA band
    qa = image.select('BQA')
    # Both bits should be zero for clear conditions
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0).And(
        qa.bitwiseAnd(clouds_bit_mask).eq(0)
    )
    return image.updateMask(mask)

def getLandsatCollection():
    """
    Merges Landsat 5, 7, 8, and 9 collections (Tier 1, Collection 2 SR) 
    and standardizes the band names for consistent analysis.

    Returns:
        ee.ImageCollection: A merged collection of standardized Landsat images.
    """
    # Define the band mappings for each Landsat version
    bn9 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
    bn8 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
    bn7 = ['SR_B1', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
    bn5 = ['SR_B1', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
    # Standardized names for all bands
    standard_bands = ['uBlue', 'Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2', 'BQA']

    # Fetch and rename bands in the Landsat collections
    ls5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").select(bn5, standard_bands)
    ls7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterDate('1999-04-15', '2003-05-30').select(bn7, standard_bands)
    ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").select(bn8, standard_bands)
    ls9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").select(bn9, standard_bands)

    # Merge all collections
    merged_collection = ls5.merge(ls7).merge(ls8).merge(ls9)

    return merged_collection

def rescale(image):
    """
    Rescale the reflectance values of Landsat imagery to allow for use of Landsat Collection 2 in DSWE.

    Parameters:
    image (ee.Image): The input image with bands to be rescaled.

    Returns:
    ee.Image: The image with rescaled bands added.
    """
    bns = ['uBlue', 'Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2', 'BQA']
    optical_bands = image.select(bns).multiply(0.0000275).add(-0.2)
    return image.addBands(optical_bands, None, True)

## Function to import study area polygon/s or reaches.

In [4]:
def load_watershed_shapefile(folder_name, base_path):
    """
    Loads a watershed shapefile or reach shapefile into an Earth Engine FeatureCollection.

    Parameters:
        - folder_name: str, the name of the specific watershed/reach folder/shapefile. This can include one or more features.
        - base_path: str, optional base directory containing watershed folders (default set to your path).

    Returns:
        - ee.FeatureCollection: An Earth Engine FeatureCollection representing the watershed geometry.
    """
    # Construct the full path to the shapefile
    shapefile_path = os.path.join(base_path, folder_name, f"{folder_name}.shp")
    
    # Check if the shapefile exists
    if not os.path.exists(shapefile_path):
        print(f"No shapefile found at {shapefile_path}")
        return None
    
    # Load the shapefile into a GeoDataFrame
    try:
        gdf = gpd.read_file(shapefile_path)
    except Exception as e:
        print(f"Error loading shapefile: {e}")
        return None
    
    # Initialize the Earth Engine API if not already done
    if not ee.data._credentials:
        ee.Initialize()
    
    # Convert the GeoDataFrame to a GeoJSON-like structure and then to a FeatureCollection
    try:
        feature_collection = geemap.geopandas_to_ee(gdf)
    except Exception as e:
        print(f"Error converting GeoDataFrame to FeatureCollection: {e}")
        return None

    return feature_collection

## Functions to generate DSWE water masks for the provided study/area or reaches

In [5]:
# Function to generate a water mask for a given year and polygon feature
def get_water_mask_for_feature(year, max_level, polygon, percentile=50):
    """
    Generate a water mask for a given year and polygon feature using Landsat imagery,
    then reduce across time using a percentile (default = 50 for median).

    Parameters:
    year (int): The year for which to generate the water mask.
    max_level (float): The maximum DSWE water level threshold for classification.
    polygon (ee.Geometry.Polygon): The polygon feature defining the area of interest.
    percentile (int): The percentile to use for temporal reduction (e.g., 70 for more confident water detection).

    Returns:
    ee.Image: The water mask image for the specified year and polygon.
    """
    imagery = (getLandsatCollection()
               .map(maskL8sr)
               .map(rescale)
               .filterDate(f'{year}-01-01', f'{year}-12-31')
               .filterBounds(polygon))

    def classify(image):
        water_mask = ClassifyWaterJones2019(image, max_level)
        return water_mask.rename('water_mask')

    # Classify and stack binary water masks
    binary_masks = imagery.map(classify)
    stacked = binary_masks.toBands()

    # Reduce with percentile and apply threshold
    reduced = stacked.reduce(ee.Reducer.percentile([percentile])).clip(polygon)
    final_mask = reduced.rename('water_mask')

    return final_mask

# Load river reach shapefiles and merge them
def load_and_merge_reaches(csv_path):
    """
    Load river reach shapefiles specified in a CSV file and merge them into a single GeoDataFrame.

    Parameters:
    csv_path (str): The file path to the CSV file containing river data, including directory paths 
                    and river names.

    Returns:
    tuple:
        - gpd.GeoDataFrame: A GeoDataFrame containing the merged shapefiles for all river reaches.
        - str: The directory path of the GRWL (Global River Widths from Landsat) data, 
               extracted from the CSV file.
    """
    # Load the CSV file
    river_data = pd.read_csv(csv_path)
    
    # List to hold individual reach GeoDataFrames
    gdfs = []
    
    # Iterate through each row of the CSV
    for _, row in river_data.iterrows():
        # Build the full path to the reach shapefile
        working_directory = row['working_directory']
        reach_base_path = os.path.join(working_directory, "RiverMapping", "Reaches")
        os.makedirs(reach_base_path, exist_ok=True)
        river_name = row['river_name']
        shapefile_path = os.path.join(reach_base_path, river_name, f"{river_name}.shp")
        
        # Load the reach shapefile as a GeoDataFrame
        gdf = gpd.read_file(shapefile_path)
        gdfs.append(gdf)
    
    # Merge all individual GeoDataFrames into a single GeoDataFrame
    reach_merge = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    
    # Create local path for GRWL directory
    working_directory = river_data['working_directory'].iloc[0]
    grwl_path = os.path.join(working_directory, "Allen_Pavelsky_2018", "GRWL_vector_V01.01", "GRWL_vector_V01.01")
    
    # Return the merged GeoDataFrame and the GRWL directory path
    return reach_merge, grwl_path

# Convert Google Drive link to direct download link
def convert_drive_to_download_url(drive_url):
    """
    Convert a Google Drive shareable link into a direct download link.

    Parameters:
    drive_url (str): The Google Drive shareable URL for the file.

    Returns:
    str: A direct download URL for the file hosted on Google Drive.
    """
    # Extract the file ID from the Google Drive URL
    file_id = drive_url.split("/d/")[1].split("/")[0]
    
    # Construct the direct download URL using the file ID
    download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    
    return download_url

# Download and extract shapefiles from Google Drive
def download_and_extract_shapefiles(download_url, extract_folder="shapefile_data"):
    """
    Download a zip file from a given URL and extract its contents to a specified folder.

    Parameters:
    download_url (str): The direct download URL for the zip file containing shapefiles.
    extract_folder (str, optional): The folder where the extracted files will be saved. 
                                    Defaults to "shapefile_data".

    Returns:
    str: The path to the folder containing the extracted files.

    Raises:
    Exception: If the download fails (non-200 status code).
    """
    # Send a GET request to download the file
    response = requests.get(download_url)
    
    if response.status_code == 200:
        # Extract the contents of the zip file
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            z.extractall(extract_folder)
            extracted_files = z.namelist()
            print(f"Extracted files: {extracted_files}")
        
        # Return the path to the extraction folder
        return extract_folder
    else:
        # Raise an exception if the download fails
        raise Exception(f"Failed to download the file. Status code: {response.status_code}")

# Load shapefile from a specified directory
def load_shapefile(directory):
    """
    Search a directory for a shapefile (.shp) and load it into a GeoDataFrame.

    Parameters:
    directory (str): The path to the directory where the shapefile is expected to be located.

    Returns:
    gpd.GeoDataFrame: A GeoDataFrame representing the loaded shapefile.

    Raises:
    Exception: If no shapefile is found in the specified directory.
    """
    # Traverse the directory to locate shapefiles
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".shp"):
                # Construct the full path to the shapefile
                shapefile_path = os.path.join(root, file)
                
                # Load the shapefile as a GeoDataFrame
                gdf = gpd.read_file(shapefile_path)
                return gdf
    
    # Raise an exception if no shapefile is found
    raise Exception("No shapefile found in the specified directory.")

# Reproject 'tiles' to match 'reaches' CRS and find intersections
def reproject_and_intersect(tiles_gdf, reaches):
    """
    Reproject a GeoDataFrame to match the coordinate reference system (CRS) of another GeoDataFrame
    and compute the intersection to identify unique tile IDs.

    Parameters:
    tiles_gdf (gpd.GeoDataFrame): A GeoDataFrame containing tile geometries with their associated CRS.
    reaches (gpd.GeoDataFrame): A GeoDataFrame containing river reach geometries with their associated CRS.

    Returns:
    list: A list of unique tile IDs (`TILE_ID`) from the intersection of `tiles_gdf` and `reaches`.

    """
    # Reproject tiles GeoDataFrame to match the CRS of reaches GeoDataFrame
    tiles_reprojected = tiles_gdf.to_crs(reaches.crs)
    
    # Compute the intersection of the two GeoDataFrames
    intersection = gpd.overlay(reaches, tiles_reprojected, how='intersection')
    
    # Return the list of unique TILE_IDs from the intersection
    return intersection['TILE_ID'].unique().tolist()

# Load and merge matching shapefiles from a directory based on 'tile_ids'
def load_matching_shapefiles(directory, tile_ids):
    """
    Load shapefiles from a directory that match specified tile IDs and merge them into a single GeoDataFrame.

    Parameters:
    directory (str): The path to the directory where the shapefiles are located.
    tile_ids (list): A list of tile IDs to match against the shapefile names.

    Returns:
    gpd.GeoDataFrame: A merged GeoDataFrame containing all matching shapefiles.

    Raises:
    Exception: If no matching shapefiles are found in the directory.
    """
    gdfs = []  # List to store individual GeoDataFrames
    
    # Traverse the directory to find matching shapefiles
    for root, dirs, files in os.walk(directory):
        for file in files:
            for tile_id in tile_ids:
                # Check if the file starts with a tile ID and ends with .shp
                if file.startswith(tile_id) and file.endswith(".shp"):
                    shapefile_path = os.path.join(root, file)
                    
                    # Load the shapefile and append it to the list
                    gdf = gpd.read_file(shapefile_path)
                    gdfs.append(gdf)
                    print(f"Loaded shapefile: {file}")
    
    # Raise an exception if no matching shapefiles are found
    if not gdfs:
        raise Exception("No matching shapefiles found.")
    
    # Merge all loaded GeoDataFrames into a single GeoDataFrame
    merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)
    
    return merged_gdf

# gets final grwl subset
def grwl_main(csv_path, drive_url):
    """
    Main function to process river reach shapefiles and GRWL (Global River Widths from Landsat) data.

    This function integrates multiple steps:
    1. Loading and merging river reach shapefiles from a CSV file.
    2. Converting a Google Drive URL to a direct download link.
    3. Downloading and extracting tile shapefiles from the direct download link.
    4. Loading the extracted tiles as a GeoDataFrame.
    5. Finding intersecting tile IDs between tiles and reaches.
    6. Loading and merging GRWL shapefiles that match the intersecting tile IDs.

    Parameters:
    csv_path (str): The file path to the CSV file containing river reach metadata and directories.
    drive_url (str): The Google Drive shareable URL for the tiles shapefile zip archive.

    Returns:
    gpd.GeoDataFrame: A GeoDataFrame containing the merged GRWL shapefiles corresponding 
                      to the intersecting tile IDs.
    """
    # Step 1: Load and merge river reach shapefiles
    reach_merge, grwl_directory = load_and_merge_reaches(csv_path)
    
    # Step 2: Convert Drive URL to direct download
    download_url = convert_drive_to_download_url(drive_url)
    
    # Step 3: Download and extract the tiles shapefiles
    extracted_folder = download_and_extract_shapefiles(download_url)
    
    # Step 4: Load the extracted shapefile as GeoDataFrame
    tiles_gdf = load_shapefile(extracted_folder)
    
    # Step 5: Find intersecting 'TILE_ID' values between 'tiles' and 'reaches'
    tile_ids = reproject_and_intersect(tiles_gdf, reach_merge)
    
    # Step 6: Load and merge matching shapefiles from GRWL data
    grwl_gdf = load_matching_shapefiles(grwl_directory, tile_ids)
    
    # Return the final merged GeoDataFrame
    return grwl_gdf

def get_utm_epsg(longitude, latitude):
    """
    Determine the appropriate UTM (Universal Transverse Mercator) EPSG code 
    for a given geographic location based on its longitude and latitude.

    Parameters:
    longitude (float): The longitude of the location in decimal degrees.
    latitude (float): The latitude of the location in decimal degrees.

    Returns:
    int: The EPSG code corresponding to the UTM zone for the given location. 
         EPSG codes for the northern hemisphere start with 32600, and those 
         for the southern hemisphere start with 32700.
    """
    # Calculate the UTM zone based on longitude
    utm_zone = int((longitude + 180) / 6) + 1
    
    # Determine the EPSG code based on the latitude (hemisphere)
    if latitude >= 0:
        epsg_code = 32600 + utm_zone  # UTM zones for northern hemisphere
    else:
        epsg_code = 32700 + utm_zone  # UTM zones for southern hemisphere
    
    return epsg_code

def raster_to_gdf(labeled_array, transform, crs=None):
    """
    Convert a labeled raster array into a GeoDataFrame, where each unique labeled region 
    in the raster is represented as a polygon.

    Parameters:
    labeled_array (numpy.ndarray): A 2D array where each unique value represents a labeled region.
                                    Regions with a value of 0 are treated as background and ignored.
    transform (affine.Affine): The affine transformation matrix for converting pixel coordinates 
                               to geographic coordinates.
    crs (str or dict, optional): The coordinate reference system for the output GeoDataFrame. 
                                 If provided, it is assigned to the GeoDataFrame.

    Returns:
    gpd.GeoDataFrame: A GeoDataFrame containing the polygons representing each labeled region 
                      and their corresponding labels.
                      - Columns:
                        - `geometry`: The Shapely geometry (polygon) for each region.
                        - `label`: The label (value) corresponding to each region.
    """
    polygons, labels = [], []
    
    # Extract shapes and values from the labeled raster array
    for geom, value in rasterio.features.shapes(labeled_array, transform=transform):
        if value != 0:  # Ignore background regions (value 0)
            polygons.append(shape(geom))  # Convert the raster geometry to Shapely geometry
            labels.append(value)         # Store the corresponding label value
    
    # Create a GeoDataFrame from the extracted polygons and labels
    gdf = gpd.GeoDataFrame({'geometry': polygons, 'label': labels})
    
    # Assign the CRS to the GeoDataFrame, if provided
    if crs is not None:
        gdf = gdf.set_crs(crs)
    
    return gdf

def find_intersecting_pixels(raster_gdf, buff_gdf):
    """
    Identify unique pixel labels from a raster GeoDataFrame that intersect with 
    the geometries in a buffer GeoDataFrame.

    Parameters:
    raster_gdf (gpd.GeoDataFrame): A GeoDataFrame representing raster data, 
                                   where each geometry corresponds to a pixel region 
                                   and includes a `label` column for pixel values.
    buff_gdf (gpd.GeoDataFrame): A GeoDataFrame containing buffer geometries to check 
                                 for intersections with the raster geometries.

    Returns:
    set: A set of unique pixel labels (from the `label` column of `raster_gdf`) that 
         intersect with the buffer geometries.
    """
    # Perform a spatial join to find intersecting geometries
    intersections = gpd.sjoin(raster_gdf, buff_gdf, how='inner', predicate='intersects')
    
    # Extract unique labels from the intersecting geometries
    unique_labels = set(intersections['label'])
    
    return unique_labels

def filter_and_reclassify_raster(classified_raster, unique_labels, min_pixels=200):
    """
    Filter and reclassify a classified raster to include only specified components 
    and remove components smaller than a minimum pixel threshold.

    Parameters:
    classified_raster (numpy.ndarray): A 2D array where each value represents a classified region (label).
    unique_labels (set): A set of labels to retain in the final raster. 
                         Pixels with these labels will be included in the output.
    min_pixels (int, optional): The minimum number of pixels a component must have to be retained. 
                                Components smaller than this size will be removed. Defaults to 200.

    Returns:
    numpy.ndarray: A 2D array where:
                   - Retained regions are marked with `1`.
                   - Background and removed regions are marked with `0`.
    """
    # Initialize the final raster with zeros (background)
    final_raster = np.zeros_like(classified_raster, dtype=np.int32)
    
    # Retain only the specified labels in the raster
    for unique_label in unique_labels:
        final_raster[classified_raster == unique_label] = 1

    # Define a 3x3 structure for connected component labeling
    structure = np.ones((3, 3), dtype=int)
    
    # Label connected components in the filtered raster
    labeled_array, num_features = label(final_raster, structure=structure)

    # Calculate the size of each labeled component
    component_sizes = np.bincount(labeled_array.ravel())
    
    # Remove components smaller than the specified size threshold
    for component_label, size in enumerate(component_sizes):
        if size < min_pixels:
            final_raster[labeled_array == component_label] = 0

    return final_raster

def export_raster(raster, transform, crs, output_path):
    """
    Export a raster array to a GeoTIFF file with LZW compression.
    
    Parameters:
        raster (numpy array): The raster data to export.
        transform (Affine): Affine transform for the raster.
        crs (CRS): Coordinate reference system of the raster.
        output_path (str): Path to save the output GeoTIFF file.
    """
    
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=raster.shape[0],
        width=raster.shape[1],
        count=1,  # Assuming single-band raster
        dtype=raster.dtype,
        crs=crs,
        transform=transform,
        compress='LZW'  # Apply LZW compression to reduce file size
    ) as dst:
        dst.write(raster, 1)

    print(f"Raster successfully saved with compression to: {output_path}")

def cleaner_main(raster_path, grwl, reach, min_pixels, output_path):
    """
    The main function to clean a raster by filtering and reclassifying it based on GRWL data 
    and reach geometry, ultimately exporting a cleaned raster.

    Parameters:
    raster_path (str): Path to the input raster file (e.g., a water mask).
    grwl (gpd.GeoDataFrame): GeoDataFrame containing GRWL (Global River Widths from Landsat) data.
    reach (gpd.GeoDataFrame): GeoDataFrame containing the river reach geometry.
    min_pixels (int): The minimum number of pixels a component must have to be retained.
    output_path (str): Path to save the cleaned and processed raster.

    Returns:
    None: The cleaned raster is exported to the specified output path.

    Workflow:
    1. Load the raster file and extract its data, transform, and CRS.
    2. Reproject the reach geometry to match the CRS of the GRWL data.
    3. Clip and dissolve the GRWL data to the extent of the reach.
    4. Determine the appropriate UTM zone for projection and reproject the dissolved geometry.
    5. Create a buffer around the projected GRWL geometry based on the mean width.
    6. Label connected components in the raster and convert them to a GeoDataFrame.
    7. Reproject the raster GeoDataFrame to match the buffer's CRS.
    8. Identify intersecting pixels between the buffer and the raster.
    9. Filter and reclassify the raster based on intersecting pixels and minimum pixel size.
    10. Export the cleaned raster to the specified output path.
    """
    # Step 1: Load raster
    with rasterio.open(raster_path) as src:
        water_mask = src.read(1)  # Read the first band (water mask)
        transform = src.transform  # Get the raster's affine transform
        crs = src.crs  # Get the raster's CRS
    
    # Step 2: Reproject the reach to match GRWL CRS
    reach = reach.to_crs(grwl.crs)
    
    # Step 3: Clip and dissolve the GRWL data to the extent of the reach
    grwl_reach = gpd.clip(grwl, reach)
    grwl_reach_dissolved = grwl_reach.dissolve()

    # Step 4: Determine UTM zone and reproject the dissolved geometry
    centroid = grwl_reach_dissolved.geometry.centroid.iloc[0]
    longitude, latitude = centroid.x, centroid.y
    utm_epsg = get_utm_epsg(longitude, latitude)
    grwl_reach_projected = grwl_reach_dissolved.to_crs(epsg=utm_epsg)

    # Step 5: Buffer the projected GRWL geometry based on the mean width
    mean_width = grwl_reach['width_m'].mean()
    grwl_buff = grwl_reach_projected.buffer(mean_width/2)

    # Step 6: Label connected components in the raster
    structure = np.ones((3, 3), dtype=int)  # 8-connectivity
    labeled_array, num_features = label(water_mask, structure=structure)

    # Step 7: Convert labeled components to a GeoDataFrame
    mask_gdf = raster_to_gdf(labeled_array, transform, crs=crs)
    mask_gdf = mask_gdf.to_crs(grwl_buff.crs)

    # Step 8: Convert the buffer to a GeoDataFrame
    grwl_buff_gdf = gpd.GeoDataFrame(geometry=grwl_buff, crs=grwl_buff.crs)

    # Step 9: Find intersecting pixel labels
    unique_pix = find_intersecting_pixels(mask_gdf, grwl_buff_gdf)

    # Step 10: Filter and reclassify the raster based on intersecting pixels
    final_raster = filter_and_reclassify_raster(labeled_array, unique_pix, min_pixels=min_pixels)

    # Step 11: Export the cleaned raster to the specified output path
    export_raster(final_raster, transform, crs, output_path)

    print(f"Processed raster saved to: {output_path}")

# Function to export masks for a range of years and features within a polygon feature collection
def export_masks(folder_name, max_level, grwl_gdf, year_range="All", reach_range="All",
                 base_path=r'C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\Reaches', 
                 local_output_path=r'C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks',
                 min_pix=0, 
                 percentile = 50):
    """
    Generates water masks for a given river and specified range of years and reaches.
    If export fails due to size, the image is tiled and exported in smaller parts,
    and the tiles are stitched back into a single .tif file.

    Parameters:
        year_range (str, int, tuple, optional): Range of years (start, end), a single year, or "All".
        reach_range (str, int, tuple, optional): Range of reaches (start, end), a single reach, or "All".
        folder_name (str): Name of the folder containing watershed shapefile.
        max_level (int): Maximum DSWE level for water mask generation.
        base_path (str, optional): Base path to the folder containing watershed shapefiles.
        local_output_path (str, optional): Local path where the output masks will be saved.
        min_pix (int, optional): Minimum pixel count for post-processing.
        min_tile_size_km2 (float, optional): Minimum tile size in square kilometers.

    Outputs:
        Exports water masks as TIFF files to the specified local output path.
    """
    min_tile_size_km2=400

    def get_geometry_area_km2(geom):
        """Calculate the area of a given ee.Geometry in square kilometers."""
        return geom.area().divide(1e6).getInfo()  # Convert from m² to km²

    def tile_geometry(geometry, tile_size_km2):
        """Tiles the geometry into smaller parts based on the target tile size."""
        area_km2 = get_geometry_area_km2(geometry)
        if area_km2 <= tile_size_km2:
            return [geometry]  # No need to tile, small enough

        # Determine how many tiles we need based on the area
        num_tiles = math.ceil(area_km2 / tile_size_km2)
        tile_count_per_side = math.ceil(math.sqrt(num_tiles))  # Number of tiles per side

        # Get the bounding box and tile the region
        bounds = geometry.bounds()
        bbox_coords = bounds.coordinates().getInfo()[0]
        min_lon, min_lat = bbox_coords[0]
        max_lon, max_lat = bbox_coords[2]

        lon_step = (max_lon - min_lon) / tile_count_per_side
        lat_step = (max_lat - min_lat) / tile_count_per_side

        tiles = []
        for i in range(tile_count_per_side):
            for j in range(tile_count_per_side):
                tile = ee.Geometry.Rectangle([
                    [min_lon + i * lon_step, min_lat + j * lat_step],
                    [min_lon + (i + 1) * lon_step, min_lat + (j + 1) * lat_step]
                ])
                tiles.append(tile)
        return tiles

    def export_tile(tile_geom, output_file_path, year, feature_id, percentile):
        """Attempts to export a single tile, retrying if necessary."""
        for retry in range(1):  # Retry one time for network or transient errors
            try:
                water_mask = get_water_mask_for_feature(year, max_level, tile_geom, percentile)
                water_mask_reprojected = water_mask.reproject(crs=target_crs, scale=30)

                geemap.ee_export_image(
                    water_mask_reprojected,
                    filename=output_file_path,
                    scale=30,
                    region=tile_geom.getInfo()['coordinates'],
                    file_per_band=False
                )
                
                # Wait for the export to complete and check if file exists
                time.sleep(10)
                if os.path.exists(output_file_path):
                    print(f"RAW: Successfully exported {output_file_path}.")
                    return True
                else:
                    print(f"Export failed: File {output_file_path} not found.")
            except Exception as e:
                error_message = str(e)
                print(f"Failed to export {output_file_path} on retry {retry + 1}. Error: {error_message}")
                time.sleep(5)  # Delay before retrying
                
                # Check if the error is related to missing bands or invalid image data
                if "Image.select" in error_message or "no bands" in error_message:
                    print(f"Skipping tiling for {year}, reach {feature_id} due to image generation error.")
                    return "invalid_image"  # Return specific error message if image is invalid
        return "size_limit"  # Return specific message if it fails due to size

    def stitch_tiles(output_folder, reach_id, year, output_file_name):
        """Stitches multiple tiles back into a single .tif using Rasterio, applies compression, and deletes the tiles after successful stitching."""
        # Find all .tif tiles in the output folder that match the reach, year, and include 'tile' in the file name
        tile_files = glob.glob(os.path.join(output_folder, f"*reach_{reach_id}_{year}_tile_*.tif"))

        if len(tile_files) > 1:
            # Open the tile files and merge them using Rasterio
            tile_datasets = [rasterio.open(tile) for tile in tile_files]
            mosaic, out_transform = merge(tile_datasets)

            # Define the output path for the stitched file
            merged_output_path = os.path.join(output_folder, output_file_name)
            print(f"Stitching tiles into {merged_output_path}...")

            # Copy the metadata from one of the tiles (all tiles should have similar metadata)
            out_meta = tile_datasets[0].meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": out_transform,
                "compress": "LZW"  # Apply LZW compression
            })

            # Write the mosaic to a new .tif file with compression
            with rasterio.open(merged_output_path, "w", **out_meta) as dest:
                dest.write(mosaic)

            # Close all tile datasets
            for ds in tile_datasets:
                ds.close()

            print(f"Tiles successfully stitched into {merged_output_path}.")

            # Delete the tiles after successful stitching
            for tile in tile_files:
                try:
                    os.remove(tile)
                    print(f"Deleted tile: {tile}")
                except OSError as e:
                    print(f"Error deleting tile {tile}: {e}")

        elif len(tile_files) == 1:
            print("Only one tile found, no stitching needed.")
        else:
            print("No tiles found for stitching.")

    # Load watershed shapefile
    polygon_fc = load_watershed_shapefile(folder_name, base_path)

    # Define the target CRS
    target_crs = 'EPSG:4326'

    # Determine the year range
    if year_range == "All":
        year_start = 1984
        year_end = 2025
    elif isinstance(year_range, int):
        year_start = year_range
        year_end = year_range
    elif isinstance(year_range, str):
        # Check if the string is in the format "(YYYY, YYYY)"
        if re.match(r'^\(\d{4}, \d{4}\)$', year_range):  # Match the pattern (YYYY, YYYY)
            try:
                # Convert the string to a tuple of integers
                year_range = ast.literal_eval(year_range)
                year_start, year_end = year_range
            except (ValueError, SyntaxError):
                raise ValueError(f"Invalid year range format: {year_range}")
        else:
            raise ValueError(f"Invalid string format for year_range: {year_range}")
    elif isinstance(year_range, tuple) and len(year_range) == 2:
        year_start, year_end = year_range
    else:
        raise ValueError("year_range must be 'All', an int, or a tuple (start, end).")

    # Determine the reach range
    if reach_range == "All":
        reach_start = 1
        reach_end = 9999
    elif isinstance(reach_range, int):
        reach_start = reach_range
        reach_end = reach_range
    elif isinstance(reach_range, str):
        # Check if the string is in the format "(XX, YY)"
        if re.match(r'^\(\d{1,4}, \d{1,4}\)$', reach_range):  # Match (XX, YY) with 1 to 4 digits
            try:
                # Convert the string to a tuple of integers
                reach_range = ast.literal_eval(reach_range)
                reach_start, reach_end = reach_range
            except (ValueError, SyntaxError):
                raise ValueError(f"Invalid reach range format: {reach_range}")
        else:
            raise ValueError(f"Invalid string format for reach_range: {reach_range}")
    elif isinstance(reach_range, tuple) and len(reach_range) == 2:
        reach_start, reach_end = reach_range
    else:
        raise ValueError("reach_range must be 'All', an int, or a tuple (start, end).")

    # Loop through each feature in the FeatureCollection
    for feature in polygon_fc.getInfo()['features']:

        # Get the geometry and ID of the feature
        feature_geom = ee.Feature(feature).geometry()
        feature_id = int(feature['properties']['ds_order'])

        # Check if the feature_id is within the reach range
        if reach_start <= feature_id <= reach_end:
            # Create the reach subfolder if it does not exist
            reach_folder_path = os.path.join(local_output_path, folder_name, f"reach_{feature_id}", "Raw")
            if not os.path.exists(reach_folder_path):
                os.makedirs(reach_folder_path)

            for year in range(year_start, year_end + 1):
                # Set the output file path based on the folder, year, and feature ID
                output_file_name = f"{folder_name}_reach_{feature_id}_{year}_DSWE_level_{max_level}.tif"
                output_file_path = os.path.join(reach_folder_path, output_file_name)

                # Attempt to export the full water mask
                try:
                    export_status = export_tile(feature_geom, output_file_path, year, feature_id, percentile)

                    if export_status == "invalid_image":
                        print(f"Skipping further processing for {year}, reach {feature_id} due to invalid image.")
                        continue  # Skip to the next feature/year if invalid image

                    elif export_status == "size_limit":
                        # If the export failed due to size, tile the geometry and retry
                        print(f"Export failed for {output_file_name}, attempting tiling...")
                        tiles = tile_geometry(feature_geom, min_tile_size_km2)

                        tile_paths = []
                        for tile_idx, tile in enumerate(tiles):
                            tile_output_name = f"{folder_name}_reach_{feature_id}_{year}_tile_{tile_idx}_DSWE_level_{max_level}.tif"
                            tile_output_path = os.path.join(reach_folder_path, tile_output_name)
                            tile_paths.append(tile_output_path)
                            if not export_tile(tile, tile_output_path, year, feature_id, percentile):
                                print(f"Failed to export tile {tile_idx} for {output_file_name}.")
                        
                        # After tiling, stitch the tiles back together
                        stitch_tiles(reach_folder_path, feature_id, year, output_file_name)
                    
                    # Only proceed if the export was successful
                    process_folder_path = os.path.join(local_output_path, folder_name, f"reach_{feature_id}", "Processed")
                    if not os.path.exists(process_folder_path):
                        os.makedirs(process_folder_path)
        
                    raster_name = f"{folder_name}_reach_{feature_id}_{year}_DSWE_level_{max_level}.tif"
                    output_path = os.path.join(process_folder_path, output_file_name)
        
                    reach_name = f'{folder_name}.shp'
                    reaches = gpd.read_file(os.path.join(base_path, folder_name, reach_name))
                    reach = reaches[reaches['ds_order'] == feature_id]
        
                    cleaner_main(output_file_path, grwl_gdf, reach, min_pix, output_path)
                    print(f"PROCESSED: Exported {raster_name} to {process_folder_path}.")

                except Exception as e:
                    print(f"Failed to export {output_file_name}. Error: {e}")
                    continue  # Skip to the next iteration

def process_multiple_rivers(csv_file):
    """
    Process multiple rivers by generating river masks for each entry in the input CSV file.

    Parameters:
    csv_file (str): The file path to the CSV containing input parameters for multiple rivers.
                    Each row should specify parameters such as:
                    - River name
                    - Working directory
                    - DSWE threshold level
                    - Year range
                    - Reach range
                    - Minimum pixel size for filtering

    Workflow:
    1. Read the CSV file to get parameters for each river.
    2. Generate a GRWL (Global River Widths from Landsat) GeoDataFrame by calling `grwl_main`.
    3. Loop through each river in the CSV:
        - Extract input parameters for the river.
        - Create necessary directories for outputs.
        - Call `export_masks` to process and export the river mask for the current river.

    Returns:
    None: This function processes and exports river masks for each river specified in the CSV file.
    """
    drive_url = "https://drive.google.com/file/d/1K6x1E0mmLc0k7er4NCIeaZsTfHi2wxxI/view?usp=sharing"
    
    # Step 1: Read the CSV file containing input variables for multiple rivers
    river_data = pd.read_csv(csv_file)
    
    # Step 2: Generate the GRWL GeoDataFrame using the provided drive URL
    grwl_gdf = grwl_main(csv_file, drive_url)

    # Step 3: Loop through each row (each river) in the CSV
    for index, row in river_data.iterrows():
        # Extract necessary input values from the current CSV row
        Folder_name = row['river_name']  # Name of the river
        working_directory = row['working_directory']  # Directory for processing
        DSWE_threshold_level = row['DSWE_threshold_level']  # DSWE threshold for classification
        year_range = row['year_range']  # Year range for processing
        reach_range = row['reach_range']  # Reach range for processing
        min_pix = row['min_pixels']  # Minimum pixel size for filtering
        percentile = row["mask_percentile"]
        
        # Create necessary directories for reaches and output masks
        base_path = os.path.join(working_directory, "RiverMapping", "Reaches")
        os.makedirs(base_path, exist_ok=True)
        local_output_path = os.path.join(working_directory, "RiverMapping", "RiverMasks")
        os.makedirs(local_output_path, exist_ok=True)
        
        # Step 4: Call the function to process the river mask for the current river
        export_masks(Folder_name, 
                     DSWE_threshold_level,
                     grwl_gdf,
                     year_range, 
                     reach_range, 
                     base_path, 
                     local_output_path,
                     min_pix,
                     percentile)

## 7) Input parameters and generate masks, download to local folder:

In [6]:
csv_file_path = r"C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\Bermejo_river_datasheet.csv"
process_multiple_rivers(csv_file_path)

Extracted files: ['IMWtiles.dbf', 'IMWtiles.lyr', 'IMWtiles.prj', 'IMWtiles.sbn', 'IMWtiles.sbx', 'IMWtiles.shp', 'IMWtiles.shp.xml', 'IMWtiles.shx']
Loaded shapefile: SF20.shp
Loaded shapefile: SG20.shp
Loaded shapefile: SG21.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_10\Raw\Bermejo_reach_10_1984_DSWE_level_3.tif
RAW: Successfully exported C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_10\Raw\Bermejo_reach_10_1984_DSWE_level_3.tif.


C:\Users\huckr\AppData\Local\Temp\ipykernel_30404\205032179.py:457: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = grwl_reach_dissolved.geometry.centroid.iloc[0]


Raster successfully saved with compression to: C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_10\Processed\Bermejo_reach_10_1984_DSWE_level_3.tif
Processed raster saved to: C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_10\Processed\Bermejo_reach_10_1984_DSWE_level_3.tif
PROCESSED: Exported Bermejo_reach_10_1984_DSWE_level_3.tif to C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_10\Processed.
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_10\Raw\Bermejo_reach_10_1985_DSWE_level_3.tif
RAW: Successfully exported C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_10\Raw\Bermejo_reach_10_1985_DSWE_level_3.tif.


C:\Users\huckr\AppData\Local\Temp\ipykernel_30404\205032179.py:457: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = grwl_reach_dissolved.geometry.centroid.iloc[0]


Raster successfully saved with compression to: C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_10\Processed\Bermejo_reach_10_1985_DSWE_level_3.tif
Processed raster saved to: C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_10\Processed\Bermejo_reach_10_1985_DSWE_level_3.tif
PROCESSED: Exported Bermejo_reach_10_1985_DSWE_level_3.tif to C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_10\Processed.
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_10\Raw\Bermejo_reach_10_1986_DSWE_level_3.tif
RAW: Successfully exported C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_10\Raw\Bermejo_reach_10_1986_DSWE_level_3.tif.


...

Raster successfully saved with compression to: C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_1\Processed\Bermejo_reach_1_2023_DSWE_level_3.tif
Processed raster saved to: C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_1\Processed\Bermejo_reach_1_2023_DSWE_level_3.tif
PROCESSED: Exported Bermejo_reach_1_2023_DSWE_level_3.tif to C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_1\Processed.
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_1\Raw\Bermejo_reach_1_2024_DSWE_level_3.tif
RAW: Successfully exported C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_1\Raw\Bermejo_reach_1_2024_DSWE_level_3.tif.


C:\Users\huckr\AppData\Local\Temp\ipykernel_30404\205032179.py:457: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = grwl_reach_dissolved.geometry.centroid.iloc[0]


Raster successfully saved with compression to: C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_1\Processed\Bermejo_reach_1_2024_DSWE_level_3.tif
Processed raster saved to: C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_1\Processed\Bermejo_reach_1_2024_DSWE_level_3.tif
PROCESSED: Exported Bermejo_reach_1_2024_DSWE_level_3.tif to C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_1\Processed.
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_1\Raw\Bermejo_reach_1_2025_DSWE_level_3.tif
RAW: Successfully exported C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_1\Raw\Bermejo_reach_1_2025_DSWE_level_3.tif.


C:\Users\huckr\AppData\Local\Temp\ipykernel_30404\205032179.py:457: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = grwl_reach_dissolved.geometry.centroid.iloc[0]


Raster successfully saved with compression to: C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_1\Processed\Bermejo_reach_1_2025_DSWE_level_3.tif
Processed raster saved to: C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_1\Processed\Bermejo_reach_1_2025_DSWE_level_3.tif
PROCESSED: Exported Bermejo_reach_1_2025_DSWE_level_3.tif to C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\RiverMasks\Bermejo\reach_1\Processed.
